In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.pipeline import Pipeline


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
to_show = True
to_save = False
is_sample = False
limit_rows = 1000


In [4]:
# This data cleaning part gets ideas from https://www.kaggle.com/apryor6/detailed-cleaning-visualization-python
# HAN YIKAI takes charge of feature 1-9
# official file
train_file = '../input/train_ver2.csv'
test_file = '../input/test_ver2.csv'

out_path = '../output/'
def save_df2file(data_frame:pd.DataFrame, name:str):
    data_frame.to_csv(f'{out_path}{name}.csv')
# sns.set(rc = {'figure.figsize':(20,18)})


In [5]:
# 复用函数

# 统计缺失值的列
def cal_loss(data):
    fil = data.iloc[:,:24].isnull().any()
    loss_column = fil[fil==True].index
    res = data.loc[:, loss_column].isnull().sum()
    print(res)

In [7]:
# test_df = pd.read_csv(test_file,
#                       dtype={'sexo': str,
#                              'ind_nuevo': str,
#                              'ult_fec_cli_1t': str,
#                              'indext': str},
#                       # nrows=limit_rows,
#                       low_memory=False
#                       )

In [8]:
base_df = pd.read_csv(train_file,
                      dtype={'sexo': str,
                             'ind_nuevo': str,
                             'ult_fec_cli_1t': str,
                             'indext': str},
                      # nrows=limit_rows,
                      low_memory=False
                      )

# resample the data for saving memory and shuffle the dataset
if is_sample:
    base_df = base_df.sample(frac=0.5)
base_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 48 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   fecha_dato             object 
 1   ncodpers               int64  
 2   ind_empleado           object 
 3   pais_residencia        object 
 4   sexo                   object 
 5   age                    object 
 6   fecha_alta             object 
 7   ind_nuevo              object 
 8   antiguedad             object 
 9   indrel                 float64
 10  ult_fec_cli_1t         object 
 11  indrel_1mes            object 
 12  tiprel_1mes            object 
 13  indresi                object 
 14  indext                 object 
 15  conyuemp               object 
 16  canal_entrada          object 
 17  indfall                object 
 18  tipodom                float64
 19  cod_prov               float64
 20  nomprov                object 
 21  ind_actividad_cliente  float64
 22  renta           

In [9]:
df = base_df

In [10]:
df.describe()

,ncodpers,indrel,tipodom,cod_prov,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
count,1.364731e+07,1.361958e+07,13619574.0,1.355372e+07,1.361958e+07,1.085293e+07,1.364731e+07,1.364731e+07,1.364731e+07,1.364731e+07,...,1.364731e+07,1.364731e+07,1.364731e+07,1.364731e+07,1.364731e+07,1.364731e+07,1.364731e+07,1.363125e+07,1.363125e+07,1.364731e+07
mean,8.349042e+05,1.178399e+00,1.0,2.657147e+01,4.578105e-01,1.342543e+05,1.022912e-04,2.315475e-05,6.554837e-01,3.939238e-04,...,5.886582e-03,9.170965e-03,2.627404e-03,5.253636e-02,4.438868e-02,2.560761e-02,3.847718e-03,5.472434e-02,5.942854e-02,1.279162e-01
std,4.315650e+05,4.177469e+00,0.0,1.278402e+01,4.982169e-01,2.306202e+05,1.011340e-02,4.811883e-03,4.752103e-01,1.984361e-02,...,7.649791e-02,9.532502e-02,5.119083e-02,2.231060e-01,2.059571e-01,1.579616e-01,6.191053e-02,2.274414e-01,2.364250e-01,3.339965e-01
min,1.588900e+04,1.000000e+00,1.0,1.000000e+00,0.000000e+00,1.202730e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.528130e+05,1.000000e+00,1.0,1.500000e+01,0.000000e+00,6.871098e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,9.318930e+05,1.000000e+00,1.0,2.800000e+01,0.000000e+00,1.018500e+05,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.199286e+06,1.000000e+00,1.0,3.500000e+01,1.000000e+00,1.559560e+05,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.553689e+06,9.900000e+01,1.0,5.200000e+01,1.000000e+00,2.889440e+07,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [11]:
# modify data type to save memory
df.iloc[:, 24:] = df.iloc[:, 24:].astype(bool)

In [12]:
df.iloc[:, :24].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   fecha_dato             object 
 1   ncodpers               int64  
 2   ind_empleado           object 
 3   pais_residencia        object 
 4   sexo                   object 
 5   age                    object 
 6   fecha_alta             object 
 7   ind_nuevo              object 
 8   antiguedad             object 
 9   indrel                 float64
 10  ult_fec_cli_1t         object 
 11  indrel_1mes            object 
 12  tiprel_1mes            object 
 13  indresi                object 
 14  indext                 object 
 15  conyuemp               object 
 16  canal_entrada          object 
 17  indfall                object 
 18  tipodom                float64
 19  cod_prov               float64
 20  nomprov                object 
 21  ind_actividad_cliente  float64
 22  renta           

In [13]:
df.iloc[:, :24].isnull().sum()

# fecha_dato 日期
# ncodpers 客户代码
# ind_empleado 员工
# pais_residencia 国藉
# sexo 性别,
# age 年龄
# fecha_alta 注册日期,
# ind_nuevo 新的,
# antiguedad 古代,
# indrel 最后访问日期, 一个月indrel
# ult_fec_cli_1t  作为主要客户的日期
# indrel_1mes  1 (First/Primary customer), 2 (co-owner ),P (Potential),3 (former primary), 4(former co-owner)
# tiprel_1mes  月初客户关系类型 A (active), I (inactive), P (former customer 前客户),R (Potential 潜在客户)
# indresi 居住指数S (Yes) 银行和居住国相同 N (No) 不同国家
# indext  外国人指数, S 本国出生, N 非本国
# conyuemp 配偶指数, S 客户是员工配偶
# canal_entrada
# indfall
# tipodom
# cod_prov 省份编码
# nomprov 省份名称
# ind_actividad_cliente
# renta 家庭总收入
# segmento segmentation: 01 - VIP, 02 - Individuals 03 - college graduated

fecha_dato                      0
ncodpers                        0
ind_empleado                27734
pais_residencia             27734
sexo                        27804
age                             0
fecha_alta                  27734
ind_nuevo                   27734
antiguedad                      0
indrel                      27734
ult_fec_cli_1t           13622516
indrel_1mes                149781
tiprel_1mes                149781
indresi                     27734
indext                      27734
conyuemp                 13645501
canal_entrada              186126
indfall                     27734
tipodom                     27735
cod_prov                    93591
nomprov                     93591
ind_actividad_cliente       27734
renta                     2794375
segmento                   189368
dtype: int64

In [58]:
cal_loss(df)

Series([], dtype: float64)


In [15]:
df.canal_entrada.value_counts().shape

(162,)

In [16]:
# 删除27734条空行
df = df.drop(df[df['ind_empleado'].isnull()].index)

In [17]:
# 缺失值很少的删除行
df = df.drop(df[df['sexo'].isnull()].index)
df = df.drop(df[df['tipodom'].isnull()].index)

In [18]:
# 缺失值很多的删除列
df = df.drop('ult_fec_cli_1t', axis=1)
df = df.drop('conyuemp',axis=1)

In [19]:
# 省份名称直接删除, 省份编码缺失值用-1填充
df = df.drop('nomprov', axis=1)
df.cod_prov = df.cod_prov.fillna(-1)

In [36]:
df[df['indrel_1mes'].isna()].iloc[:,:df.shape[1]-24]

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,tiprel_1mes,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,ind_actividad_cliente,renta,segmento
3781457,2015-07-28,631189,N,ES,V,42,2015-07-13,1,0,1.0,...,NaN,S,N,NaN,N,1.0,28.0,0.0,105597.36,NaN
3865981,2015-07-28,565364,N,ES,H,48,2015-07-24,1,0,1.0,...,NaN,S,N,NaN,N,1.0,8.0,1.0,118791.66,NaN
3975350,2015-07-28,838985,N,ES,V,49,2015-07-22,1,0,1.0,...,NaN,S,N,NaN,N,1.0,7.0,0.0,172654.95,NaN
3977056,2015-07-28,843591,N,ES,V,40,2015-05-30,1,2,1.0,...,NaN,S,S,KHM,N,1.0,28.0,1.0,113016.54,02 - PARTICULARES
3979448,2015-07-28,828469,N,ES,H,48,2015-07-31,1,0,1.0,...,NaN,S,N,NaN,N,1.0,7.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13560753,2016-05-28,1250361,N,ES,V,24,2016-04-11,1,1,1.0,...,NaN,S,N,KHN,N,1.0,28.0,0.0,NaN,02 - PARTICULARES
13569088,2016-05-28,1264262,N,ES,V,45,2016-04-14,1,1,1.0,...,NaN,S,N,KHM,N,1.0,17.0,0.0,NaN,02 - PARTICULARES
13606492,2016-05-28,1121175,N,ES,V,28,2016-05-24,1,0,1.0,...,NaN,S,N,NaN,N,1.0,28.0,0.0,NaN,NaN
13621915,2016-05-28,1175116,N,ES,V,25,2016-04-12,1,1,1.0,...,NaN,S,N,KHN,N,1.0,28.0,0.0,NaN,02 - PARTICULARES


In [48]:
# 11.indrel_1mes 1 (First/Primary customer), 2 (co-owner ),P (Potential),3 (former primary), 4(former co-owner)
# 12.tiprel_1mes 月初客户关系类型 A (active), I (inactive), P (former customer 前客户),R (Potential 潜在客户)
df.indrel_1mes.value_counts()
df.indrel_1mes = df.indrel_1mes.fillna(0)
df.tiprel_1mes = df.tiprel_1mes.fillna('unknown')

1.0    9133349
1      4357263
3.0       2779
3         1570
P          873
2.0        740
2          577
4.0        223
4           83
Name: indrel_1mes, dtype: int64

In [57]:
df.tiprel_1mes.value_counts()

I          7304864
A          6187065
unknown     122047
P             4655
R              869
N                4
Name: tiprel_1mes, dtype: int64

In [56]:
# canal_entrada 用户加入渠道非常多162种 而且比较平衡, 还是用unknown填充, 考虑KNN填充
df.canal_entrada.value_counts()
df.canal_entrada =  df.canal_entrada.fillna('unknown')

KHE    4055270
KAT    3268209
KFC    3098360
KHQ     591039
KFA     409669
        ...   
KDI         17
KDL         11
025         11
KHS          5
KHR          1
Name: canal_entrada, Length: 163, dtype: int64

In [54]:
df.renta.mean

<bound method NDFrame._add_numeric_operations.<locals>.mean of 0            87218.10
1            35548.74
2           122179.11
3           119775.54
4                 NaN
              ...    
13647304     43912.17
13647305     23334.99
13647306          NaN
13647307    199592.82
13647308          NaN
Name: renta, Length: 13619504, dtype: float64>

In [55]:
df.renta.value_counts()
df.renta = df.renta.fillna(df.renta.mean)

451931.22    5582
463625.16    1743
181042.20    1493
128318.52    1404
105260.88    1354
             ... 
100303.05       1
119497.38       1
74566.20        1
78719.40        1
133142.67       1
Name: renta, Length: 520992, dtype: int64

In [ ]:
df

In [46]:
# renta 数值型填充
sns.scatterplot(df.renta, df.renta.value_counts())
# df.renta.plot.scatter(x='renta', y=df.renta.value_counts())

TypeError: scatterplot() takes from 0 to 1 positional arguments but 2 were given

In [47]:
# 23. segmento : 01 - VIP, 02 - 个人 03 - 大学毕业
df.segmento = df.segmento.fillna(00)

In [34]:
df.cod_prov.value_counts(dropna=False)

 28.0    4409547
 8.0     1275219
 46.0     682304
 41.0     605164
 15.0     429322
 30.0     396759
 29.0     367023
 50.0     342543
 3.0      313397
 11.0     294684
 36.0     280026
 33.0     265749
 47.0     238259
 35.0     235683
 6.0      192996
 48.0     185888
 45.0     183067
 18.0     178562
 37.0     164238
 39.0     155706
 14.0     144679
 10.0     129912
 7.0      124933
 21.0     122283
 13.0     119390
 2.0      114128
 12.0     104295
 43.0     100115
 9.0       97188
 17.0      90538
 31.0      88618
 26.0      85202
 27.0      84962
 32.0      84009
 24.0      83003
 25.0      79059
 20.0      71567
 38.0      70968
-1.0       65856
 19.0      64618
 23.0      64363
 4.0       60291
 16.0      57310
 49.0      50927
 34.0      49282
 40.0      42341
 22.0      40181
 5.0       38783
 1.0       37704
 44.0      22525
 42.0      17660
 52.0       9460
 51.0       7218
Name: cod_prov, dtype: int64

In [131]:
# cod_prov和nomprov 省份代号, 直接填充unknown
df.loc[:, ['cod_prov', 'nomprov','segmento','canal_entrada']] = df.loc[:, ['cod_prov', 'nomprov','segmento','canal_entrada']].fillna('unknown')

In [130]:
# canal_entrada 用户加入渠道非常多 而且比较平衡, 还是用unknown填充
df.canal_entrada.value_counts(dropna=False)


KHE    4055270
KAT    3268209
KFC    3098360
KHQ     591039
KFA     409669
        ...   
KDI         17
KDL         11
025         11
KHS          5
KHR          1
Name: canal_entrada, Length: 163, dtype: int64

In [59]:
cal_loss(df)

Series([], dtype: float64)


In [ ]:
save_df2file(df, 'filledna')